# 0. Limpeza dos Dados para a Análise

In [1]:
import pandas as pd
from haversine import haversine, Unit

df = pd.read_csv('../src/data/train.csv')
df.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,19-03-2022,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,(min) 24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,25-03-2022,19:45:00,19:50:00,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,(min) 33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,19-03-2022,08:30:00,08:45:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,(min) 26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,05-04-2022,18:00:00,18:10:00,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,(min) 21
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,26-03-2022,13:30:00,13:45:00,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,(min) 30


In [2]:
df1 = df.copy()


# 1. Conversão a coluna Age de texto para número
linhas_selecionadas = (df1 ['Delivery_person_Age'] != 'NaN ')
df1 = df1.loc[linhas_selecionadas, :].copy()
df1['Delivery_person_Age'] = df1['Delivery_person_Age']. astype( int )
df1.shape

linhas_selecionadas = (df1 ['Road_traffic_density'] != 'NaN ')
df1 = df1.loc[linhas_selecionadas, :].copy()

linhas_selecionadas = (df1 ['City'] != 'NaN ')
df1 = df1.loc[linhas_selecionadas, :].copy()

linhas_selecionadas = (df1 ['Festival'] != 'NaN ')
df1 = df1.loc[linhas_selecionadas, :].copy()


# 2. Conversão a coluna de Ratings de texto para número decimal (float)
df1['Delivery_person_Ratings'] = df1['Delivery_person_Ratings']. astype( float )

# 3. Conversão da coluna order_date de texto para data
df1['Order_Date'] = pd.to_datetime( df1['Order_Date'], format='%d-%m-%Y' )

# 4. Conversão de Multiple_deliveries para Interiro
linhas_selecionadas = (df1 ['multiple_deliveries'] != 'NaN ')
df1 = df1.loc[linhas_selecionadas, :].copy()
df1['multiple_deliveries'] = df1['multiple_deliveries'].astype( int )

# 5. Removendo os espaços dentro de strings/texto/objects
df1.loc[:, 'ID'] = df1.loc[:, 'ID'].str.strip()
df1.loc[:, 'Road_traffic_density'] = df1.loc[:, 'Road_traffic_density'].str.strip()
df1.loc[:, 'Type_of_order'] = df1.loc[:, 'Type_of_order'].str.strip()
df1.loc[:, 'Type_of_vehicle'] = df1.loc[:, 'Type_of_vehicle'].str.strip()
df1.loc[:, 'City'] = df1.loc[:, 'City'].str.strip()
df1.loc[:, 'Festival'] = df1.loc[:, 'Festival'].str.strip()


# 6. Limpeza sobre a coluna Time_taken(min)
df1['Time_taken(min)'] = df1['Time_taken(min)'].apply( lambda x: x.split( '(min) ' )[1])
df1['Time_taken(min)'] = df1['Time_taken(min)'].astype( int )

In [3]:
df1.head(5)

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,19:50:00,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,08:45:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,18:10:00,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,21
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,13:45:00,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,30


# 1. Quantidade de Pedidos por dia🔰

In [4]:
# Colunas
cols = ['ID', 'Order_Date']

# Seleção de Linhas
# Assign the result of the groupby operation to df_aux
df_aux = df1.loc[:, cols].groupby( 'Order_Date' ).count().reset_index()

# Desenho do Gráfico de Barras para representar
#ploty

import plotly.express as px
# Now df_aux is defined and can be used by px.bar
px.bar ( df_aux, x='Order_Date', y='ID')


# 2. Número de Pedidos por Semana! 🔰

In [5]:
df1 ['week_of_year']  = df1 ['Order_Date'].dt.strftime( '%U' )
df_aux = df1.loc[:, ['ID', 'week_of_year']].groupby( 'week_of_year' ).count().reset_index()
px.line(df_aux, x='week_of_year', y='ID')

# 3. Distribuição por pedidos por tipo de tráfego 🔰

In [6]:
df_aux = df1.loc[:, ['ID', 'Road_traffic_density']].groupby('Road_traffic_density').count().reset_index()
df_aux = df_aux.loc[df_aux['Road_traffic_density'] != 'NaN', :]
df_aux['entregas_perc'] = df_aux['ID'] / df_aux['ID'].sum()
px.pie(df_aux, values='ID', names='Road_traffic_density')

# 4. Comparação do Volume de pedidos por cidade e tipo de tráfego. 🔰

In [7]:
df_aux = df1.loc[:,['ID', 'City', 'Road_traffic_density']].groupby(['City', 'Road_traffic_density']).count().reset_index()
df_aux = df_aux.loc[df_aux['City'] != 'NaN', :]
df_aux = df_aux.loc[df_aux['Road_traffic_density'] != 'NaN', :]

px.scatter(df_aux, x='City', y='Road_traffic_density', size='ID', color='City')

# 5. Quantidade de pedidos entregue por semana! 🔰

In [8]:
df_aux01 = df1.loc[:, ['ID', 'week_of_year']].groupby('week_of_year').count().reset_index()
df_aux02 = df1.loc[:, ['Delivery_person_ID', 'week_of_year']].groupby('week_of_year').nunique().reset_index()

df_aux = pd.merge(df_aux01, df_aux02, how='inner')
df_aux['order_by_deliver'] = df_aux['ID'] / df_aux['Delivery_person_ID']

px.line(df_aux, x='week_of_year', y='order_by_deliver')

# 6. Localização Central de cada cidade por tipo de tráfego!🔰

In [9]:
import folium

df_aux = (df.loc[:, ['City','Road_traffic_density','Delivery_location_latitude', 'Delivery_location_longitude']]
          .groupby(['City', 'Road_traffic_density'])
          .median().reset_index())

df_aux = df_aux.loc[df_aux['City'] != 'NaN', :]
df_aux = df_aux.loc[df_aux['Road_traffic_density'] != 'NaN', :]
df_aux = df_aux.head(50)
map = folium.Map()

for index, location_info in df_aux.iterrows():
  folium.Marker([location_info ['Delivery_location_latitude'],
                 location_info ['Delivery_location_longitude']],
                popup=location_info[['City', 'Road_traffic_density']]).add_to(map)

map

# **1. A menor e maior idade dos entregadores 👀**

In [10]:
# A menor idade dos Entregadores

print('A maior idade é:{}' .format (df1.loc[:,'Delivery_person_Age'].max()))
print('A menor idade é:{}' .format (df1.loc[:,'Delivery_person_Age'].min()))

A maior idade é:39
A menor idade é:20


# **2. A pior e a melhor condição de veículos 👀**


In [11]:
# A pior e a melhor condição de veículos 👀

print('A melhor condição é:{}' .format (df1.loc[:,'Vehicle_condition'].max()))
print('A pior condição é :{}'  .format (df1.loc[:,'Vehicle_condition'].min()))

A melhor condição é:2
A pior condição é :0


# **3. A avaliação média por entregador 👀**

In [12]:
df_avg_ratings_per_deliver = df1.loc[:, ['Delivery_person_ID', 'Delivery_person_Ratings']].groupby('Delivery_person_ID').mean().reset_index()
df_avg_ratings_per_deliver

,Delivery_person_ID,Delivery_person_Ratings
0,AGRRES010DEL01,4.761538
1,AGRRES010DEL02,4.671429
2,AGRRES010DEL03,4.575000
3,AGRRES01DEL01,4.522222
4,AGRRES01DEL02,4.700000
...,...,...
1315,VADRES19DEL02,4.632727
1316,VADRES19DEL03,4.670270
1317,VADRES20DEL01,4.620370
1318,VADRES20DEL02,4.591111


# **4. Avaliação média e o desvio padrão por tipo de tráfego 👀**

In [13]:

df1['Delivery_person_Ratings'] = pd.to_numeric(df1['Delivery_person_Ratings'], errors='coerce')
df1 = df1.dropna(subset=['Delivery_person_Ratings'])

df_avg_std_rating_by_traffic = (
    df1.loc[:, ['Delivery_person_Ratings', 'Road_traffic_density']]
    .groupby('Road_traffic_density')
    .agg({'Delivery_person_Ratings': ['mean', 'std']})
)

# Renomeando as colunas
df_avg_std_rating_by_traffic.columns = ['delivery_mean', 'delivery_std']

# Resetando o índice
df_avg_std_rating_by_traffic = df_avg_std_rating_by_traffic.reset_index()

print(df_avg_std_rating_by_traffic)


  Road_traffic_density  delivery_mean  delivery_std
0                 High       4.652230      0.273044
1                  Jam       4.594019      0.329778
2                  Low       4.645011      0.338080
3               Medium       4.660138      0.274245


# **5. Avaliação média e o desvio padrão por condições climáticas 👀**

In [14]:

df1['Delivery_person_Ratings'] = pd.to_numeric(df1['Delivery_person_Ratings'], errors='coerce')
df1 = df1.dropna(subset=['Delivery_person_Ratings'])

df_avg_std_rating_by_traffic = (
    df1.loc[:, ['Delivery_person_Ratings', 'Weatherconditions']]
    .groupby('Weatherconditions')
    .agg({'Delivery_person_Ratings': ['mean', 'std']})
)

# Renomeando as colunas
df_avg_std_rating_by_traffic.columns = ['delivery_mean', 'delivery_std']

# Resetando o índice
df_avg_std_rating_by_traffic = df_avg_std_rating_by_traffic.reset_index()

print(df_avg_std_rating_by_traffic)

       Weatherconditions  delivery_mean  delivery_std
0      conditions Cloudy       4.651871      0.281197
1         conditions Fog       4.652965      0.275060
2  conditions Sandstorms       4.611748      0.310852
3      conditions Stormy       4.611819      0.313096
4       conditions Sunny       4.654868      0.396674
5       conditions Windy       4.616128      0.304565


# **6. Os 10 entregadores mais rápidos por cidade 👀**

In [15]:
df2 = df1.loc[:, ['Delivery_person_ID', 'City', 'Time_taken(min)']
            ].groupby(['City','Delivery_person_ID']).min().sort_values(['City','Time_taken(min)'],
            ascending=True).reset_index()

df_aux01 = df2.loc[df2['City']=='Metropolitian', :].head(10)
df_aux02 = df2.loc[df2['City']=='Urban', :].head(10)
df_aux03 = df2.loc[df2['City']=='Semi-Urban	', :].head(10)

df3 = pd.concat([df_aux01, df_aux02, df_aux03]).reset_index(drop=True)

df3



,City,Delivery_person_ID,Time_taken(min)
0,Metropolitian,AGRRES010DEL03,10
1,Metropolitian,AGRRES07DEL03,10
2,Metropolitian,AGRRES12DEL01,10
3,Metropolitian,AGRRES14DEL01,10
4,Metropolitian,AGRRES17DEL03,10
5,Metropolitian,ALHRES08DEL03,10
6,Metropolitian,ALHRES12DEL01,10
7,Metropolitian,ALHRES14DEL02,10
8,Metropolitian,ALHRES15DEL02,10
9,Metropolitian,ALHRES19DEL03,10


In [16]:
df1.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,...,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min),week_of_year
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,...,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,24,11
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,...,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,33,12
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,...,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,26,11
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,...,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,21,14
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,...,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,30,12


# **7. Os 10 entregadores mais lentos por cidade 👀**

In [17]:
df2 = df1.loc[:, ['Delivery_person_ID', 'City', 'Time_taken(min)']
            ].groupby(['City','Delivery_person_ID']).max().sort_values(['City','Time_taken(min)'],
            ascending=False).reset_index()

df_aux01 = df2.loc[df2['City']=='Metropolitian', :].head(10)
df_aux02 = df2.loc[df2['City']=='Urban', :].head(10)
df_aux03 = df2.loc[df2['City']=='Semi-Urban	', :].head(10)

df3 = pd.concat([df_aux01, df_aux02, df_aux03]).reset_index(drop=True)

df3


,City,Delivery_person_ID,Time_taken(min)
0,Metropolitian,ALHRES010DEL01,54
1,Metropolitian,AURGRES13DEL01,54
2,Metropolitian,BANGRES02DEL01,54
3,Metropolitian,BHPRES16DEL02,54
4,Metropolitian,CHENRES02DEL02,54
5,Metropolitian,CHENRES04DEL01,54
6,Metropolitian,CHENRES07DEL03,54
7,Metropolitian,CHENRES08DEL01,54
8,Metropolitian,CHENRES13DEL02,54
9,Metropolitian,COIMBRES010DEL03,54


# **1. A quantidade média dos restaurantes e dos locais de entrega 🐍💕**

In [18]:
delivery_unique =  len(df1.loc[:, 'Delivery_person_ID'].unique())

print('Onúmero de entregadores únicos é:{}'.format(delivery_unique))

Onúmero de entregadores únicos é:1320


# **2. A distância média dos restaurantes e dos locais de entrega 🐍💕**

In [19]:
cols = ['Delivery_location_latitude', 'Delivery_location_longitude', 'Restaurant_latitude', 'Restaurant_longitude']
df1['distance'] = df1.loc[:, cols].apply(lambda x: haversine(
                                        (x['Restaurant_latitude'], x['Restaurant_longitude']),
                                        (x['Delivery_location_latitude'], x['Delivery_location_longitude'])), axis=1 )

avg_distance = df1['distance'].mean()
print('A distância média das entregas é de: {:.2f} KM'.format(avg_distance))


A distância média das entregas é de: 26.91 KM


In [20]:
import plotly.graph_objects as go

cols = ['Delivery_location_latitude', 'Delivery_location_longitude', 'Restaurant_latitude', 'Restaurant_longitude']
df1['distance'] = df1.loc[:, cols].apply(lambda x: haversine(
                                        (x['Restaurant_latitude'], x['Restaurant_longitude']),
                                        (x['Delivery_location_latitude'], x['Delivery_location_longitude'])), axis=1 )

avg_distance = df1.loc[:,[ 'City', 'distance']].groupby('City').mean().reset_index()
fig = go.Figure ( data = 
                 [go.Pie ( labels = avg_distance ['City'], 
                          values=avg_distance ['distance'], 
                          pull = [0.01, 0.01, 0.01])])
fig.show()



# **3. O tempo médio eo desvio padrão de entrega por cidade 🐍💕**

In [21]:
import plotly.graph_objects as go

cols = ['City', 'Time_taken(min)']

df_aux = df1.loc[:, cols].groupby('City').agg({'Time_taken(min)': ['mean', 'std']})

df_aux.columns = ['avg_time', 'std_time']

df_aux = df_aux.reset_index()

fig = go.Figure()
fig.add_trace(go.Bar(name='Control',
                     x=df_aux['City'],
                     y=df_aux['avg_time'],
                     error_y=dict(type='data', array=df_aux['std_time'])))

fig.update_layout(barmode='group')
fig.show()
                                                   

# **4. O tempo médio e o desvio padrão de entrega por cidade e tipo de pedido. 🐍💕**

In [22]:
cols = ['City', 'Time_taken(min)','Type_of_order']

df_aux = df1.loc [:, cols].groupby (['City','Type_of_order'] ).agg ({'Time_taken(min)': ['mean', 'std']})
df_aux.columns = ['avg_time', 'std_time']                                                     
df_aux = df_aux.reset_index()
df_aux


,City,Type_of_order,avg_time,std_time
0,Metropolitian,Buffet,27.301935,9.155644
1,Metropolitian,Drinks,27.324962,9.041698
2,Metropolitian,Meal,27.618877,9.215624
3,Metropolitian,Snack,27.467867,9.119682
4,Semi-Urban,Buffet,49.707317,2.731702
5,Semi-Urban,Drinks,49.625000,2.459347
6,Semi-Urban,Meal,50.300000,3.041665
7,Semi-Urban,Snack,49.408163,2.707385
8,Urban,Buffet,23.566337,9.060933
9,Urban,Drinks,23.308885,8.925721


# **5. O tempo médio e o desvio padrão de entrega por cidade e tipo de tráfego. 🐍💕**

In [23]:
cols = ['City', 'Time_taken(min)','Road_traffic_density']

df_aux = df1.loc [:, cols].groupby (['City','Road_traffic_density'] ).agg ({'Time_taken(min)': ['mean', 'std']})
df_aux.columns = ['avg_time', 'std_time']                                                     
df_aux = df_aux.reset_index()
df_aux

,City,Road_traffic_density,avg_time,std_time
0,Metropolitian,High,28.145976,7.904494
1,Metropolitian,Jam,31.978193,9.477798
2,Metropolitian,Low,22.259874,6.795802
3,Metropolitian,Medium,27.728045,8.309769
4,Semi-Urban,High,50.125000,2.629956
5,Semi-Urban,Jam,49.841270,2.717095
6,Semi-Urban,Medium,47.400000,2.011080
7,Urban,High,24.305335,8.494842
8,Urban,Jam,27.989541,10.078679
9,Urban,Low,19.445964,6.319847


In [24]:
import numpy as np
import plotly.express as px

cols = ['City', 'Time_taken(min)', 'Road_traffic_density']

df_aux = df1.loc[:, cols].groupby(['City', 'Road_traffic_density']).agg({'Time_taken(min)': ['mean', 'std']})
df_aux.columns = ['avg_time', 'std_time']
df_aux = df_aux.reset_index()

fig = px.sunburst(df_aux, 
                  path=['City', 'Road_traffic_density'], 
                  values='avg_time',
                  color='std_time', 
                  color_continuous_scale='RdBu',
                  color_continuous_midpoint=np.average(df_aux['std_time']))

fig.show()



# **6. O tempo médio de entrega durantes os Festivais. 🐍💕**

In [25]:
cols = ['Time_taken(min)','Festival']

df_aux = df1.loc [:, cols].groupby (['Festival'] ).agg ({'Time_taken(min)': ['mean', 'std']})
df_aux.columns = ['avg_time', 'std_time']                                                     
df_aux = df_aux.reset_index()


linhas_selecionadas = df_aux['Festival'] == 'Yes'
df_aux = df_aux.loc[linhas_selecionadas, :]

df_aux 









,Festival,avg_time,std_time
1,Yes,45.518607,4.005399


In [26]:
df1.head(10)


,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,...,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min),week_of_year,distance
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,...,High,2,Snack,motorcycle,0,No,Urban,24,11,3.025153
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,...,Jam,2,Snack,scooter,1,No,Metropolitian,33,12,20.183558
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,...,Low,0,Drinks,motorcycle,1,No,Urban,26,11,1.552760
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,...,Medium,0,Buffet,motorcycle,1,No,Metropolitian,21,14,7.790412
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,...,High,1,Snack,scooter,1,No,Metropolitian,30,12,6.210147
5,0x9bb4,HYDRES09DEL03,22,4.8,17.431668,78.408321,17.461668,78.438321,2022-03-11,21:20:00,...,Jam,0,Buffet,motorcycle,1,No,Urban,26,10,4.610372
6,0x95b4,RANCHIRES15DEL01,33,4.7,23.369746,85.339820,23.479746,85.449820,2022-03-04,19:15:00,...,Jam,1,Meal,scooter,1,No,Metropolitian,40,09,16.600384
7,0x9eb2,MYSRES15DEL02,35,4.6,12.352058,76.606650,12.482058,76.736650,2022-03-14,17:25:00,...,Medium,2,Meal,motorcycle,1,No,Metropolitian,32,11,20.205281
8,0x1102,HYDRES05DEL02,22,4.8,17.433809,78.386744,17.563809,78.516744,2022-03-20,20:55:00,...,Jam,0,Buffet,motorcycle,1,No,Metropolitian,34,12,19.975548
9,0xcdcd,DEHRES17DEL01,36,4.2,30.327968,78.046106,30.397968,78.116106,2022-02-12,21:55:00,...,Jam,2,Snack,motorcycle,3,No,Metropolitian,46,06,10.280596
